In [1]:
#!pip install plotly.express

In [2]:
%config Complete.use_jedi = False

In [3]:
import pandas as pd
import numpy as np
import json

In [4]:
profile = pd.read_json("./starbucks_data/profile.json", lines = True)
portfolio = pd.read_json("./starbucks_data/portfolio.json", lines=True)
transcript = pd.read_json("./starbucks_data/transcript.json", lines=True)

In [5]:
# getting basic info about the files
print(profile.info())
print(portfolio.info())
print(transcript.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes
None
<class 'panda

In [6]:
#na values for gender and income

In [7]:
print(profile.describe())
print(portfolio.describe())
print(transcript.describe())

                age  became_member_on         income
count  17000.000000      1.700000e+04   14825.000000
mean      62.531412      2.016703e+07   65404.991568
std       26.738580      1.167750e+04   21598.299410
min       18.000000      2.013073e+07   30000.000000
25%       45.000000      2.016053e+07   49000.000000
50%       58.000000      2.017080e+07   64000.000000
75%       73.000000      2.017123e+07   80000.000000
max      118.000000      2.018073e+07  120000.000000
          reward  difficulty   duration
count  10.000000   10.000000  10.000000
mean    4.200000    7.700000   6.500000
std     3.583915    5.831905   2.321398
min     0.000000    0.000000   3.000000
25%     2.000000    5.000000   5.000000
50%     4.000000    8.500000   7.000000
75%     5.000000   10.000000   7.000000
max    10.000000   20.000000  10.000000
                time
count  306534.000000
mean      366.382940
std       200.326314
min         0.000000
25%       186.000000
50%       408.000000
75%       528.00

In [8]:
#max_values for age may be corrupted
#min values for time may be a measurement_error from the backend

In [9]:
profile_df = profile.groupby("gender").size().reset_index(name="amount")
profile_df["percentage"]=(profile_df.amount/profile_df.amount.sum())*100
profile_df
#skewed toward males

,gender,amount,percentage
0,F,6129,41.342327
1,M,8484,57.227656
2,O,212,1.430017


In [10]:
profile.groupby(["gender"]).mean()
# on the contrary female have much more income than the males , and females are older (maybe signifcantly, but we do not know for sure)

,age,became_member_on,income
gender,,,
F,57.544950,2.016704e+07,71306.412139
M,52.116690,2.016675e+07,61194.601603
O,54.400943,2.016790e+07,63287.735849


In [11]:
unique_channels = set()
for item in portfolio.channels:
    for channel in item:
        unique_channels.add(channel)
print(unique_channels)
#4 unique media channels

{'email', 'web', 'mobile', 'social'}


In [12]:
portfolio = portfolio.explode("channels").reset_index(drop=True)
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,email,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,mobile,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
2,10,social,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
3,10,web,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
4,10,email,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
5,10,mobile,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
6,10,social,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
7,0,web,0,4,informational,3f207df678b143eea3cee63160fa8bed
8,0,email,0,4,informational,3f207df678b143eea3cee63160fa8bed
9,0,mobile,0,4,informational,3f207df678b143eea3cee63160fa8bed


In [13]:
portfolio_small = portfolio.groupby(["channels"]).size().reset_index(name="amount")
portfolio_small["percentage"]=(portfolio_small.amount/portfolio_small.amount.sum())*100
portfolio_small
#social is the less frequent channel, email the most

,channels,amount,percentage
0,email,10,30.303030
1,mobile,9,27.272727
2,social,6,18.181818
3,web,8,24.242424


In [14]:
portfolio.groupby(["offer_type", "channels"]).mean().sort_values("difficulty")
#informational is always the least difficulty, 
# highest mean difficulty is for discount and email/web
# informational has the least duration in contrast to discount
# bogo has the highest reward in the mean especially for social
#discount has a moderate reward but the highest difficulty, users need very long to fulfill these, because they need a higher amount, this can be correlated?
#difficulty minimum required spend to complete an offer, so informational costs nothing but can bring something.that would require an uplift scenario...
#bogo has the same difficulty as the reward which makes sense because it is buy one get one free

reward  difficulty  duration
offer_type    channels                                
informational email     0.000000    0.000000  3.500000
              mobile    0.000000    0.000000  3.500000
              social    0.000000    0.000000  3.000000
              web       0.000000    0.000000  4.000000
bogo          web       6.666667    6.666667  5.666667
              email     7.500000    7.500000  6.000000
              mobile    7.500000    7.500000  6.000000
              social    8.333333    8.333333  5.666667
discount      social    2.500000    8.500000  8.500000
              mobile    2.333333    9.000000  8.000000
              email     3.000000   11.750000  8.500000
              web       3.000000   11.750000  8.500000

In [15]:
portfolio.groupby(["offer_type", "channels"]).max().sort_values("duration")

reward  difficulty  duration  \
offer_type    channels                                 
informational social         0           0         3   
              email          0           0         4   
              mobile         0           0         4   
              web            0           0         4   
bogo          email         10          10         7   
              mobile        10          10         7   
              social        10          10         7   
              web           10          10         7   
discount      email          5          20        10   
              mobile         3          10        10   
              social         3          10        10   
              web            5          20        10   

                                                      id  
offer_type    channels                                    
informational social    5a8bc65990b245e5a138643cd4eb9837  
              email     5a8bc65990b245e5a138643cd4eb9837  
              mobile    5a8bc65990b245e5a138643cd4eb9837  
              web       3f207df678b143eea3cee63160fa8bed  
bogo          email     f19421c1d4aa40978ebb69ca19b0e20d  
              mobile    f19421c1d4aa40978ebb69ca19b0e20d  
              social    f19421c1d4aa40978ebb69ca19b0e20d  
              web       f19421c1d4aa40978ebb69ca19b0e20d  
discount      email     fafdcd668e3743c1bb461111dcafc2a4  
              mobile    fafdcd668e3743c1bb461111dcafc2a4  
              social    fafdcd668e3743c1bb461111dcafc2a4  
              web       fafdcd668e3743c1bb461111dcafc2a4

In [16]:
portfolio.groupby(["offer_type", "channels"]).min().sort_values("duration")

reward  difficulty  duration  \
offer_type    channels                                 
informational email          0           0         3   
              mobile         0           0         3   
              social         0           0         3   
              web            0           0         4   
bogo          email          5           5         5   
              mobile         5           5         5   
              social         5           5         5   
              web            5           5         5   
discount      email          2           7         7   
              mobile         2           7         7   
              social         2           7         7   
              web            2           7         7   

                                                      id  
offer_type    channels                                    
informational email     3f207df678b143eea3cee63160fa8bed  
              mobile    3f207df678b143eea3cee63160fa8bed  
              social    5a8bc65990b245e5a138643cd4eb9837  
              web       3f207df678b143eea3cee63160fa8bed  
bogo          email     4d5c57ea9a6940dd891ad53e9dbe8da0  
              mobile    4d5c57ea9a6940dd891ad53e9dbe8da0  
              social    4d5c57ea9a6940dd891ad53e9dbe8da0  
              web       4d5c57ea9a6940dd891ad53e9dbe8da0  
discount      email     0b1e1539f2cc45b7b9fa7c272da2e1d7  
              mobile    2298d6c36e964ae4a3e7e9706d1fb8c2  
              social    2298d6c36e964ae4a3e7e9706d1fb8c2  
              web       0b1e1539f2cc45b7b9fa7c272da2e1d7

In [17]:
#min max values vary much more for reward, difficuly and duration, when you are not informational
# one business question, do we get an uplift by informational only
# is discount better than bogo or vice versa?
# do we have channel-dependent effects in this
# are their certain group of users who react more or less to the certain combinations

In [18]:
df_trans = transcript.groupby("event").size().reset_index(name="amount")
df_trans

,event,amount
0,offer completed,33579
1,offer received,76277
2,offer viewed,57725
3,transaction,138953


In [19]:
df_trans["percentage"] = (df_trans.amount/df_trans.amount.sum())*100
df_trans
#transaction has a specific amount spended, while the others only have the id of an offer e.g.

,event,amount,percentage
0,offer completed,33579,10.954413
1,offer received,76277,24.883700
2,offer viewed,57725,18.831516
3,transaction,138953,45.330371


In [20]:
# does transaction has an offer pendant?
transcript.person.duplicated().values.any()

True

In [21]:
transcript[transcript.person.duplicated()].sort_values("person")

,person,event,value,time
168412,0009655768c64bdeb2e877511632db8f,transaction,{'amount': 8.57},414
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372
247879,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},576
168413,0009655768c64bdeb2e877511632db8f,offer completed,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...,414
187554,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},456
...,...,...,...,...
133074,ffff82501cea40309d5fdd7edcca4a07,offer viewed,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},354
274809,ffff82501cea40309d5fdd7edcca4a07,transaction,{'amount': 10.12},606
200255,ffff82501cea40309d5fdd7edcca4a07,transaction,{'amount': 13.17},498
143788,ffff82501cea40309d5fdd7edcca4a07,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,384


In [22]:
#further explorative data would be great if data would be joined
# values are unique per id, thus a grouping per offer id is useful
#for this we are cleaning every data frame for merging

In [23]:
portfolio["email"] = np.where(portfolio.channels == "email", 1, 0)
portfolio["mobile"] = np.where(portfolio.channels == "mobile", 1, 0)
portfolio["social"] = np.where(portfolio.channels == "social", 1, 0)
portfolio["web"] = np.where(portfolio.channels == "web", 1, 0)
portfolio_cleaned = portfolio.groupby("id").agg(
    {"reward": "max", 
     "difficulty": "max", 
     "duration": "max", 
     "offer_type":"max",
     "email": "max",
     "social": "max",
     "web": "max",
     "mobile": "max"}
).reset_index()

In [24]:
portfolio_cleaned

,id,reward,difficulty,duration,offer_type,email,social,web,mobile
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,discount,1,0,1,0
1,2298d6c36e964ae4a3e7e9706d1fb8c2,3,7,7,discount,1,1,1,1
2,2906b810c7d4411798c6938adc9daaa5,2,10,7,discount,1,0,1,1
3,3f207df678b143eea3cee63160fa8bed,0,0,4,informational,1,0,1,1
4,4d5c57ea9a6940dd891ad53e9dbe8da0,10,10,5,bogo,1,1,1,1
5,5a8bc65990b245e5a138643cd4eb9837,0,0,3,informational,1,1,0,1
6,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1,0,1,1
7,ae264e3637204a6fb9bb56bc8210ddfd,10,10,7,bogo,1,1,0,1
8,f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,bogo,1,1,1,1
9,fafdcd668e3743c1bb461111dcafc2a4,2,10,10,discount,1,1,1,1


In [25]:
profile.gender = np.where(profile.gender.isna(), np.nan, profile.gender)
# put all None and Nan on the same value

In [26]:
profile_cleaned = profile
profile_cleaned

,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,M,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [27]:
transcript.value.isna().values.any()

False

In [28]:
type(transcript.value[0])

dict

In [29]:
#check max length of dict for unfolding
# seems to be 2
for k, v in transcript.value.items():
    if len(v) > 1:
        print(v)
        break
# remove break to see that everything is ok

{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2}


In [30]:
# we can delete the second part, as the offer_ids already have a reward colum in the portfolio table, this would make unfolding easier
for k, v in transcript.value.items():
    if len(v) > 1:
        v = v.pop("reward")

In [31]:
#check again
for k, v in transcript.value.items():
    if len(v) > 1:
        print(v)       
#empty!

In [32]:
transcript_trans = transcript[transcript.event == "transaction"]
transcript_trans.value = [v2 for k,v in transcript_trans.value.items() for k2, v2 in v.items()]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [33]:
transcript_trans["coupon_code"] = 0

<ipython-input-33-18cd839adec3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcript_trans["coupon_code"] = 0


In [34]:
transcript_offers = transcript[transcript.event != "transaction"]
transcript_offers

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},714
306506,b895c57e8cd047a8872ce02aa54759d6,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},714
306507,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer viewed,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},714
306509,8431c16f8e1d440880db371a68f82dd0,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},714


In [35]:
transcript_offers.value = [v2 for k, v in transcript_offers.value.items() for k2, v2 in v.items()]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [36]:
transcript_offers.rename(columns={"value": "coupon_code"}, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [37]:
transcript_offers["value"] = 0

<ipython-input-37-68a0ef04b1e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcript_offers["value"] = 0


In [38]:
transcript_offers.append(transcript_trans)

,person,event,coupon_code,time,value
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,0.00
1,a03223e636434f42ac4c3df47e8bac43,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,0.00
2,e2127556f4f64592b11af22de27a7932,offer received,2906b810c7d4411798c6938adc9daaa5,0,0.00
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,fafdcd668e3743c1bb461111dcafc2a4,0,0.00
4,68617ca6246f4fbc85e91a2a49552598,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,0,0.00
...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,0,714,1.59
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,0,714,9.53
306531,a00058cf10334a308c68e7631c529907,transaction,0,714,3.61
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,0,714,3.53


In [39]:
transcript #looking at head and tail, seems equal and is cleaned

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


In [40]:
transcript_cleaned = transcript_offers.append(transcript_trans)

In [41]:
transcript_cleaned

,person,event,coupon_code,time,value
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,0.00
1,a03223e636434f42ac4c3df47e8bac43,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,0.00
2,e2127556f4f64592b11af22de27a7932,offer received,2906b810c7d4411798c6938adc9daaa5,0,0.00
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,fafdcd668e3743c1bb461111dcafc2a4,0,0.00
4,68617ca6246f4fbc85e91a2a49552598,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,0,0.00
...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,0,714,1.59
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,0,714,9.53
306531,a00058cf10334a308c68e7631c529907,transaction,0,714,3.61
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,0,714,3.53


In [42]:
# merging person with profile, should be 1:1 every person should have one profile (may be incorrect, but in theory should be ok
# every person gets n offers as can be seen in the person data. Every coupon has one set of charasterics, so also 1:1.

In [43]:
transcript_cleaned.rename(columns={"person": "id"}, inplace=True)
transcript_cleaned_merged = transcript_cleaned.merge(profile_cleaned, how="left")
portfolio_cleaned.rename(columns={"id": "coupon_code"}, inplace=True)
final_df = transcript_cleaned_merged.merge(portfolio_cleaned, how="left")
final_df.sort_values("id")

,id,event,coupon_code,time,value,gender,age,became_member_on,income,reward,difficulty,duration,offer_type,email,social,web,mobile
130750,0009655768c64bdeb2e877511632db8f,offer completed,fafdcd668e3743c1bb461111dcafc2a4,528,0.00,M,33,20170421,72000.0,2.0,10.0,10.0,discount,1.0,1.0,1.0,1.0
30049,0009655768c64bdeb2e877511632db8f,offer received,5a8bc65990b245e5a138643cd4eb9837,168,0.00,M,33,20170421,72000.0,0.0,0.0,3.0,informational,1.0,1.0,0.0,1.0
139458,0009655768c64bdeb2e877511632db8f,offer received,2906b810c7d4411798c6938adc9daaa5,576,0.00,M,33,20170421,72000.0,2.0,10.0,7.0,discount,1.0,0.0,1.0,1.0
105987,0009655768c64bdeb2e877511632db8f,offer viewed,f19421c1d4aa40978ebb69ca19b0e20d,456,0.00,M,33,20170421,72000.0,5.0,5.0,5.0,bogo,1.0,1.0,1.0,1.0
270873,0009655768c64bdeb2e877511632db8f,transaction,0,552,13.56,M,33,20170421,72000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48236,ffff82501cea40309d5fdd7edcca4a07,offer completed,0b1e1539f2cc45b7b9fa7c272da2e1d7,198,0.00,F,45,20161125,62000.0,5.0,20.0,10.0,discount,1.0,0.0,1.0,0.0
152513,ffff82501cea40309d5fdd7edcca4a07,offer viewed,2906b810c7d4411798c6938adc9daaa5,582,0.00,F,45,20161125,62000.0,2.0,10.0,7.0,discount,1.0,0.0,1.0,1.0
199012,ffff82501cea40309d5fdd7edcca4a07,transaction,0,198,22.88,F,45,20161125,62000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111129,ffff82501cea40309d5fdd7edcca4a07,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,504,0.00,F,45,20161125,62000.0,5.0,5.0,7.0,bogo,1.0,0.0,1.0,1.0


In [44]:
# now we have to shape the data a little bit more, to make ti a little bit more easier, it would be good if we would have one person per line
# there may be other approaches, but most of the time it helps to have one object of interest per row or one timestamp per row or a object * timestamp. 
#Here we have several events per person in several lines. It may be that a person with more offers may distort a ML method by the amount of rows 

In [45]:
final_df["offer_received"] = np.where(final_df.event == "offer received", 1, 0)
final_df["offer_viewed"] = np.where(final_df.event == "offer viewed", 1, 0)
final_df["offer_completed"] = np.where(final_df.event == "offer completed", 1, 0)
final_df["transaction"] = np.where(final_df.event == "transaction", 1, 0)
final_df["offer_views_missed"] = final_df["offer_received"] - final_df["offer_viewed"] # should be 0 to have no offers missed
final_df.sort_values("id")

,id,event,coupon_code,time,value,gender,age,became_member_on,income,reward,...,offer_type,email,social,web,mobile,offer_received,offer_viewed,offer_completed,transaction,offer_views_missed
130750,0009655768c64bdeb2e877511632db8f,offer completed,fafdcd668e3743c1bb461111dcafc2a4,528,0.00,M,33,20170421,72000.0,2.0,...,discount,1.0,1.0,1.0,1.0,0,0,1,0,0
30049,0009655768c64bdeb2e877511632db8f,offer received,5a8bc65990b245e5a138643cd4eb9837,168,0.00,M,33,20170421,72000.0,0.0,...,informational,1.0,1.0,0.0,1.0,1,0,0,0,1
139458,0009655768c64bdeb2e877511632db8f,offer received,2906b810c7d4411798c6938adc9daaa5,576,0.00,M,33,20170421,72000.0,2.0,...,discount,1.0,0.0,1.0,1.0,1,0,0,0,1
105987,0009655768c64bdeb2e877511632db8f,offer viewed,f19421c1d4aa40978ebb69ca19b0e20d,456,0.00,M,33,20170421,72000.0,5.0,...,bogo,1.0,1.0,1.0,1.0,0,1,0,0,-1
270873,0009655768c64bdeb2e877511632db8f,transaction,0,552,13.56,M,33,20170421,72000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48236,ffff82501cea40309d5fdd7edcca4a07,offer completed,0b1e1539f2cc45b7b9fa7c272da2e1d7,198,0.00,F,45,20161125,62000.0,5.0,...,discount,1.0,0.0,1.0,0.0,0,0,1,0,0
152513,ffff82501cea40309d5fdd7edcca4a07,offer viewed,2906b810c7d4411798c6938adc9daaa5,582,0.00,F,45,20161125,62000.0,2.0,...,discount,1.0,0.0,1.0,1.0,0,1,0,0,-1
199012,ffff82501cea40309d5fdd7edcca4a07,transaction,0,198,22.88,F,45,20161125,62000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0
111129,ffff82501cea40309d5fdd7edcca4a07,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,504,0.00,F,45,20161125,62000.0,5.0,...,bogo,1.0,0.0,1.0,1.0,1,0,0,0,1


In [46]:
# now we do this for the offer type
final_df["informational"] = np.where(final_df.offer_type == "informational", 1, 0)
final_df["bogo"] = np.where(final_df.offer_type == "bogo", 1, 0)
final_df["discount"] = np.where(final_df.offer_type == "discount", 1, 0)

In [47]:
# now we have to deal with time, time says: time in hours since start of test. The data begins at time t=0, so we can see when it was the first time a user got an offer, and how long he needed between 
# individual steps to take a final transaction per different offer_type. 

#Now we have to keep in mind our business goals. Do we want to predict, what revenue the consumer will generate with which offer?
#which offer was more beneficial?  how long should be the time intervalls between each offer? Did the user even VIEWED the offer and then completed it? or did he just completed it?
#Every question needs a different analysis and
#changing the data
final_df.sort_values("id").head(20).sort_values("time")

# we will try an intermediate solution to cover more or less all of these aspects, although this may be not an ideal solution, we will be able to 
# keep a lot of the ideas here. So we bascially are at the step of feature engineering
# we will take per person: min time -> thus first time it was addressed, >max time, last time it was adressed. 
# mean value between offer received, offer reviewed, offer completed and transaction.
# a chain of interactions that shows if a person hit 4 on a row or missed one step

#min and max will be made with the grouping

,id,event,coupon_code,time,value,gender,age,became_member_on,income,reward,...,web,mobile,offer_received,offer_viewed,offer_completed,transaction,offer_views_missed,informational,bogo,discount
30049,0009655768c64bdeb2e877511632db8f,offer received,5a8bc65990b245e5a138643cd4eb9837,168,0.00,M,33,20170421,72000.0,0.0,...,0.0,1.0,1,0,0,0,1,1,0,0
47320,0009655768c64bdeb2e877511632db8f,offer viewed,5a8bc65990b245e5a138643cd4eb9837,192,0.00,M,33,20170421,72000.0,0.0,...,0.0,1.0,0,1,0,0,-1,1,0,0
205213,0009655768c64bdeb2e877511632db8f,transaction,0,228,22.16,M,33,20170421,72000.0,NaN,...,NaN,NaN,0,0,0,1,0,0,0,0
58310,0009655768c64bdeb2e877511632db8f,offer received,3f207df678b143eea3cee63160fa8bed,336,0.00,M,33,20170421,72000.0,0.0,...,1.0,1.0,1,0,0,0,1,1,0,0
77536,0009655768c64bdeb2e877511632db8f,offer viewed,3f207df678b143eea3cee63160fa8bed,372,0.00,M,33,20170421,72000.0,0.0,...,1.0,1.0,0,1,0,0,-1,1,0,0
83625,0009655768c64bdeb2e877511632db8f,offer received,f19421c1d4aa40978ebb69ca19b0e20d,408,0.00,M,33,20170421,72000.0,5.0,...,1.0,1.0,1,0,0,0,1,0,1,0
239045,0009655768c64bdeb2e877511632db8f,transaction,0,414,8.57,M,33,20170421,72000.0,NaN,...,NaN,NaN,0,0,0,1,0,0,0,0
96948,0009655768c64bdeb2e877511632db8f,offer completed,f19421c1d4aa40978ebb69ca19b0e20d,414,0.00,M,33,20170421,72000.0,5.0,...,1.0,1.0,0,0,1,0,0,0,1,0
105987,0009655768c64bdeb2e877511632db8f,offer viewed,f19421c1d4aa40978ebb69ca19b0e20d,456,0.00,M,33,20170421,72000.0,5.0,...,1.0,1.0,0,1,0,0,-1,0,1,0
112507,0009655768c64bdeb2e877511632db8f,offer received,fafdcd668e3743c1bb461111dcafc2a4,504,0.00,M,33,20170421,72000.0,2.0,...,1.0,1.0,1,0,0,0,1,0,0,1


In [48]:
final_df["last_time_offer_received"] = np.where(final_df.event == "offer received", final_df.time, 0)
final_df["last_time_offer_completed"] = np.where(final_df.event == "offer completed", final_df.time, 0)
final_df["first_time_offer_received"] = np.where(final_df.event == "offer received", final_df.time, 0)
final_df["first_time_offer_completed"] = np.where(final_df.event == "offer completed", final_df.time, 0)
# this can be used with min/max later in groupby to get more information out of it
# first offer received can be created by using the same np.where and later in the groupby make a min value instead of max for last_time offer received

In [49]:
#collected reward
#only applies if offer was completed, otherwise reward dint count per user. We are using the maximum amount of reward in sum as a total reward
final_df["total_reward_completed"] = np.where(final_df.event == "offer completed", final_df.reward, 0)
# this can be used with sum later in groupby to get more information out of it
# this could also be used for a value - reward calculation later on
# offer reward will only be counted if offer was completed

In [50]:
#duration will only be counted if offer was completed
final_df["total_duration_completed"] = np.where(final_df.event == "offer completed", final_df.duration, 0)

In [51]:
#difficulty will only be counted if offer was completed
final_df["total_difficulty_completed"] = np.where(final_df.event == "offer completed", final_df.difficulty, 0)

In [52]:
#so that we see what a user was offered in terms of difficulty and time to fulfill to compare that aainst the things he really completed
final_df.rename(columns={"difficulty": "difficulty_offered", "duration": "duration_offered"}, inplace=True)

In [53]:
final_df = final_df.sort_values(["id", "time", "coupon_code"])

In [54]:
#order may change due to database time lags, but the last two events happen more or less in the same time, 
#so catcth both we just chnage the order of both events at the end

final_df["amount_of_paths_completed_part1"] = 0
count = 0
for _ in range(len(final_df)):
    if list(final_df.event.iloc[count:_+4]) == ["offer received", "offer viewed", "transaction", "offer completed"]:
        
        final_df.amount_of_paths_completed_part1.iloc[count:_+4] = 1
    else:
        final_df.amount_of_paths_completed_part1.iloc[count:_+4] = 0
    count +=1

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [55]:
final_df["amount_of_paths_completed_part2"] = 0
count = 0
for _ in range(len(final_df)):
    if list(final_df.event.iloc[count:_+4]) == ["offer received", "offer viewed", "offer completed", "transaction"]:
        
        final_df.amount_of_paths_completed_part2.iloc[count:_+4] = 1
    else:
        final_df.amount_of_paths_completed_part2.iloc[count:_+4] = 0
    count +=1
    
final_df["amount_of_completed_user_paths"] = final_df["amount_of_paths_completed_part1"] + final_df["amount_of_paths_completed_part2"]
final_df = final_df.sort_values(["id", "time"])
final_df["mean_time_between_actions"] = final_df["time"].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
#make the mean later for groupy_by

#however that induces negative values, when the lambda function changes to the next id, we will encode therefore, all NaN and negative values with 0
final_df[final_df.mean_time_between_actions < 0]

,id,event,coupon_code,time,value,gender,age,became_member_on,income,reward,...,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,total_reward_completed,total_duration_completed,total_difficulty_completed,amount_of_paths_completed_part1,amount_of_paths_completed_part2,amount_of_completed_user_paths,mean_time_between_actions
30552,00116118485d4dfda04fdbaba9a87b5c,offer received,f19421c1d4aa40978ebb69ca19b0e20d,168,0.00,NaN,118,20180425,NaN,5.0,...,0,168,0,0.0,0.0,0.0,0,0,0,-528.0
3066,0011e0d4e6b944f998e987f904e8c1e5,offer received,3f207df678b143eea3cee63160fa8bed,0,0.00,O,40,20180109,57000.0,0.0,...,0,0,0,0.0,0.0,0.0,0,0,0,-630.0
1889,0020c2b971eb4e9188eac86d93036a77,offer received,fafdcd668e3743c1bb461111dcafc2a4,0,0.00,F,59,20160304,90000.0,2.0,...,0,0,0,0.0,0.0,0.0,1,0,1,-654.0
174232,0020ccbbb6d84e358d3414a3ff76cffd,transaction,0,42,16.27,F,24,20161111,60000.0,NaN,...,0,0,0,0.0,0.0,0.0,0,0,0,-666.0
2652,003d66b6608740288d6cc97a6903f4f0,offer received,5a8bc65990b245e5a138643cd4eb9837,0,0.00,F,26,20170621,73000.0,0.0,...,0,0,0,0.0,0.0,0.0,0,0,0,-672.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3787,fff3ba4757bd42088c044ca26d73817a,offer received,fafdcd668e3743c1bb461111dcafc2a4,0,0.00,F,69,20150920,83000.0,2.0,...,0,0,0,0.0,0.0,0.0,0,0,0,-684.0
4601,fff7576017104bcc8677a8d63322b5e1,offer received,fafdcd668e3743c1bb461111dcafc2a4,0,0.00,M,71,20171031,73000.0,2.0,...,0,0,0,0.0,0.0,0.0,0,0,0,-576.0
169944,fff8957ea8b240a6b5e634b6ee8eafcf,transaction,0,18,0.64,M,71,20180218,56000.0,NaN,...,0,0,0,0.0,0.0,0.0,0,0,0,-678.0
184,fffad4f4828548d1b5583907f2e9906b,offer received,f19421c1d4aa40978ebb69ca19b0e20d,0,0.00,M,34,20170123,34000.0,5.0,...,0,0,0,0.0,0.0,0.0,1,0,1,-660.0


In [56]:
final_df.sort_values(["id", "time"]).head(15)

,id,event,coupon_code,time,value,gender,age,became_member_on,income,reward,...,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,total_reward_completed,total_duration_completed,total_difficulty_completed,amount_of_paths_completed_part1,amount_of_paths_completed_part2,amount_of_completed_user_paths,mean_time_between_actions
30049,0009655768c64bdeb2e877511632db8f,offer received,5a8bc65990b245e5a138643cd4eb9837,168,0.00,M,33,20170421,72000.0,0.0,...,0,168,0,0.0,0.0,0.0,0,0,0,NaN
47320,0009655768c64bdeb2e877511632db8f,offer viewed,5a8bc65990b245e5a138643cd4eb9837,192,0.00,M,33,20170421,72000.0,0.0,...,0,0,0,0.0,0.0,0.0,0,0,0,24.0
205213,0009655768c64bdeb2e877511632db8f,transaction,0,228,22.16,M,33,20170421,72000.0,NaN,...,0,0,0,0.0,0.0,0.0,0,0,0,36.0
58310,0009655768c64bdeb2e877511632db8f,offer received,3f207df678b143eea3cee63160fa8bed,336,0.00,M,33,20170421,72000.0,0.0,...,0,336,0,0.0,0.0,0.0,0,0,0,108.0
77536,0009655768c64bdeb2e877511632db8f,offer viewed,3f207df678b143eea3cee63160fa8bed,372,0.00,M,33,20170421,72000.0,0.0,...,0,0,0,0.0,0.0,0.0,0,0,0,36.0
83625,0009655768c64bdeb2e877511632db8f,offer received,f19421c1d4aa40978ebb69ca19b0e20d,408,0.00,M,33,20170421,72000.0,5.0,...,0,408,0,0.0,0.0,0.0,0,0,0,36.0
239045,0009655768c64bdeb2e877511632db8f,transaction,0,414,8.57,M,33,20170421,72000.0,NaN,...,0,0,0,0.0,0.0,0.0,0,0,0,6.0
96948,0009655768c64bdeb2e877511632db8f,offer completed,f19421c1d4aa40978ebb69ca19b0e20d,414,0.00,M,33,20170421,72000.0,5.0,...,414,0,414,5.0,5.0,5.0,0,0,0,0.0
105987,0009655768c64bdeb2e877511632db8f,offer viewed,f19421c1d4aa40978ebb69ca19b0e20d,456,0.00,M,33,20170421,72000.0,5.0,...,0,0,0,0.0,0.0,0.0,0,0,0,42.0
112507,0009655768c64bdeb2e877511632db8f,offer received,fafdcd668e3743c1bb461111dcafc2a4,504,0.00,M,33,20170421,72000.0,2.0,...,0,504,0,0.0,0.0,0.0,0,0,0,48.0


In [57]:
final_df.mean_time_between_actions = \
np.where((final_df.mean_time_between_actions < 0) | np.isnan(final_df.mean_time_between_actions), 0, final_df.mean_time_between_actions)
final_df.sort_values(["id", "time"]).head(15) 
#works!

,id,event,coupon_code,time,value,gender,age,became_member_on,income,reward,...,last_time_offer_completed,first_time_offer_received,first_time_offer_completed,total_reward_completed,total_duration_completed,total_difficulty_completed,amount_of_paths_completed_part1,amount_of_paths_completed_part2,amount_of_completed_user_paths,mean_time_between_actions
30049,0009655768c64bdeb2e877511632db8f,offer received,5a8bc65990b245e5a138643cd4eb9837,168,0.00,M,33,20170421,72000.0,0.0,...,0,168,0,0.0,0.0,0.0,0,0,0,0.0
47320,0009655768c64bdeb2e877511632db8f,offer viewed,5a8bc65990b245e5a138643cd4eb9837,192,0.00,M,33,20170421,72000.0,0.0,...,0,0,0,0.0,0.0,0.0,0,0,0,24.0
205213,0009655768c64bdeb2e877511632db8f,transaction,0,228,22.16,M,33,20170421,72000.0,NaN,...,0,0,0,0.0,0.0,0.0,0,0,0,36.0
58310,0009655768c64bdeb2e877511632db8f,offer received,3f207df678b143eea3cee63160fa8bed,336,0.00,M,33,20170421,72000.0,0.0,...,0,336,0,0.0,0.0,0.0,0,0,0,108.0
77536,0009655768c64bdeb2e877511632db8f,offer viewed,3f207df678b143eea3cee63160fa8bed,372,0.00,M,33,20170421,72000.0,0.0,...,0,0,0,0.0,0.0,0.0,0,0,0,36.0
83625,0009655768c64bdeb2e877511632db8f,offer received,f19421c1d4aa40978ebb69ca19b0e20d,408,0.00,M,33,20170421,72000.0,5.0,...,0,408,0,0.0,0.0,0.0,0,0,0,36.0
239045,0009655768c64bdeb2e877511632db8f,transaction,0,414,8.57,M,33,20170421,72000.0,NaN,...,0,0,0,0.0,0.0,0.0,0,0,0,6.0
96948,0009655768c64bdeb2e877511632db8f,offer completed,f19421c1d4aa40978ebb69ca19b0e20d,414,0.00,M,33,20170421,72000.0,5.0,...,414,0,414,5.0,5.0,5.0,0,0,0,0.0
105987,0009655768c64bdeb2e877511632db8f,offer viewed,f19421c1d4aa40978ebb69ca19b0e20d,456,0.00,M,33,20170421,72000.0,5.0,...,0,0,0,0.0,0.0,0.0,0,0,0,42.0
112507,0009655768c64bdeb2e877511632db8f,offer received,fafdcd668e3743c1bb461111dcafc2a4,504,0.00,M,33,20170421,72000.0,2.0,...,0,504,0,0.0,0.0,0.0,0,0,0,48.0


In [58]:
final_df.to_csv("./starbucks_data/starbucks_df_per_coupon.csv", index=False)
#we need this later on

In [60]:
final_df_per_person = final_df.groupby("id").agg(
    {
     "id": "max",
     "time": "max", 
     "value": "sum",
     "gender": "max",
     "age": "max",
     "became_member_on": "max",
     "income": "max",
     "reward": "sum",
     "offer_received": "sum",
     "offer_viewed": "sum",
     "offer_completed": "sum",
     "transaction": "sum",
     "offer_views_missed": "sum", 
     "informational": "sum",
     "bogo": "sum",
     "discount": "sum",
     "total_reward_completed": "sum",
     "total_duration_completed": "mean",
     "total_difficulty_completed": "mean",
     "difficulty_offered": "mean",
     "duration_offered": "mean",
     "last_time_offer_received": "max",
     "last_time_offer_completed": "max",
     "first_time_offer_received": "min",
     "first_time_offer_completed": "min",
     "mean_time_between_actions": "mean",
     "amount_of_completed_user_paths": "sum",
     "email": "sum",
     "social": "sum",
     "web": "sum",
     "mobile": "sum"
    }

)

final_df_per_person.to_csv("./starbucks_data/starbucks_df_per_person.csv", index=False)